In [1]:
"""
UNet configs comparison: from batchflow, from batchflow with fix, mine
"""

'\nUNet configs comparison: from batchflow, from batchflow with fix, mine\n'

In [2]:
import sys
sys.path.append("..")
# sys.path.append('../../..')

In [3]:
import os
import shutil

# import warnings
# warnings.filterwarnings('ignore')

# from matplotlib import pyplot as plt
# %matplotlib inline

# from tensorflow import logging
# logging.set_verbosity(logging.ERROR)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
from batchflow import Pipeline, B, C, V, D#, L
from batchflow.opensets import PascalSegmentation
from batchflow.research import Research, Option, Domain, Results, PrintLogger, RP, REP, KV
from batchflow.models.metrics import Loss
from batchflow.models.torch import UNet, EncoderDecoder
# from torch.multiprocessing import set_start_method

In [4]:
# set_start_method('spawn')

In [5]:
BATCH_SIZE = 64
ITERATIONS = 1#1000
N_REPS = 1#10
IMAGE_SHAPE = (160, 160)

dataset = PascalSegmentation(bar='n')

In [6]:
def clear_previous_results(res_name):
    if os.path.exists(res_name):
        shutil.rmtree(res_name)

In [7]:
num_classes = 22

task_config = {
#     'inputs/images/shape': (3, 160, 160), # can be commented
    'inputs/targets/classes': num_classes,
#     'initial_block/inputs': 'images', # can be commented   
    'head/layout': 'c',
    'head/filters': num_classes,
    'head/kernel_size': 1,
    'loss': 'ce',
    'optimizer': 'Adam'
}

# # UNet from batchflow
# UNet_bf = UNet(task_config)

# # fixed UNet from batchflow
# config_bf_with_fix = dict(UNet_bf.full_config.config)
# config_bf_with_fix['body/decoder/upsample'] = dict(layout='tna', filters=[512, 256, 128, 64])
# UNet_bf_fix = EncoderDecoder(config_bf_with_fix)

In [8]:
# my_UNet
downsample_depth = 4

my_config = {
    'body/encoder': {
        'num_stages': downsample_depth,
        'order': ['block', 'skip', 'downsampling']
    },    
    'body/encoder/blocks': {
        'layout': 'cna cna',
        'filters': [32*pow(2, i) for i in range(1, downsample_depth+1)]
    },
    'body/encoder/downsample': {
        'layout': 'p'
    },    
    
    'body/embedding': {
        'layout': 'cna cna', 
        'filters': [64*pow(2, downsample_depth), 64*pow(2, downsample_depth)]
    },   

    'body/decoder': {
        'num_stages': downsample_depth,
        'order': ['upsampling', 'combine', 'block']
    },
    'body/decoder/upsample': {
        'layout': 'tna',
        'filters': [32*pow(2, i) for i in range(downsample_depth, -1, -1)]
    },
    'body/decoder/combine': {
        'op': 'concat'
    },
    'body/decoder/blocks': {
        'layout': 'cna cna',
        'filters': [64*pow(2, i-1) for i in range(downsample_depth, -1, -1)]
    }
}
my_config.update(task_config)
# my_UNet = EncoderDecoder(my_config)

In [9]:
my_config

{'body/encoder': {'num_stages': 4, 'order': ['block', 'skip', 'downsampling']},
 'body/encoder/blocks': {'layout': 'cna cna', 'filters': [64, 128, 256, 512]},
 'body/encoder/downsample': {'layout': 'p'},
 'body/embedding': {'layout': 'cna cna', 'filters': [1024, 1024]},
 'body/decoder': {'num_stages': 4,
  'order': ['upsampling', 'combine', 'block']},
 'body/decoder/upsample': {'layout': 'tna',
  'filters': [512, 256, 128, 64, 32]},
 'body/decoder/combine': {'op': 'concat'},
 'body/decoder/blocks': {'layout': 'cna cna',
  'filters': [512, 256, 128, 64, 32.0]},
 'inputs/targets/classes': 22,
 'head/layout': 'c',
 'head/filters': 22,
 'head/kernel_size': 1,
 'loss': 'ce',
 'optimizer': 'Adam'}

In [10]:
def process_mask(x):
    x = np.squeeze(x)
    np.place(x, x==255, 21)
    return x

train_ppl = (dataset.train.p
    .init_variable('loss_history', [])
    .init_model('dynamic', C('model'), 'model', config=C('config'))
#     .init_model('dynamic', UNet, 'model', config=task_config)
    .resize(size=IMAGE_SHAPE, src=['images', 'labels'], dst=['images', 'labels'])
    .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])
    .process_mask(B('labels'), save_to=B('labels'))
    .train_model('model', B('images'), B('labels'),
                fetches='loss', save_to=V('loss_history', mode='a'))
    .run_later(BATCH_SIZE, shuffle=True, n_epochs=None)
)



In [11]:
# train_ppl.set_config({'config': my_config, 'model': EncoderDecoder})

In [12]:
# train_ppl.next_batch()

In [13]:
# configs = [KV(task_config, "config_bf"), KV(config_bf_with_fix, "config_bf_with_fix"), KV(my_config, "my_config")]
# domain = Option('model', [UNet, UNet, EncoderDecoder]) @ Option('config', configs)
# list(domain.iterator)

configs = [KV(my_config, "my_config")]
domain = Option('model', [EncoderDecoder]) @ Option('config', configs)
list(domain.iterator)

[ConfigAlias({'model': 'EncoderDecoder', 'config': 'my_config', 'repetition': '0'})]

In [14]:
research = (Research()
            .init_domain(domain, n_reps=N_REPS)
            .add_pipeline(train_ppl, variables='loss_history', name='train_ppl', logging=True))

res_name='UNet_pascal_research'
clear_previous_results(res_name)

In [15]:
research.run(n_iters=ITERATIONS, name=res_name, bar=True, workers=3, devices=[2, 3, 4])

Research UNet_pascal_research is starting...


Domain updated: 0: : 0it [00:00, ?it/s]

<class 'numpy.ndarray'> (2, 3, 160, 160)
cuda:0
<class 'numpy.ndarray'> (64, 3, 160, 160)
cuda:0
<class 'numpy.ndarray'> (64, 160, 160)
cuda:0


Domain updated: 0: 100%|██████████| 1/1.0 [00:40<00:00, 40.32s/it]


In [16]:
results = research.load_results().df
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          1 non-null      object
 1   loss_history  1 non-null      object
 2   iteration     1 non-null      int64 
 3   sample_index  1 non-null      object
 4   model         1 non-null      object
 5   config        1 non-null      object
 6   repetition    1 non-null      int64 
 7   update        1 non-null      int64 
dtypes: int64(3), object(5)
memory usage: 192.0+ bytes


../batchflow/batchflow/research/results.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  intersection = pd.np.arange(start, end)
../batchflow/batchflow/research/results.py:111: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  elements_to_load = pd.np.array([pd.np.isin(it, iterations_to_load) for it in iterations])
../batchflow/batchflow/research/results.py:115: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  res[variable] = pd.np.array(dumped_file[variable])[elements_to_load]


In [17]:
results.head()

,name,loss_history,iteration,sample_index,model,config,repetition,update
0,train_ppl,[3.2949722],0,996987848,EncoderDecoder,my_config,0,0


In [18]:
# add test pipeline for getting metrics with param execute last